# J2: JSSP Advanced (Multi-Objective) - Multiagent Pattern


<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together.

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**J2: JSSP Advanced (Multi-Objective)** - Job-shop scheduling with multiple optimization objectives.


In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


In [ ]:
import sys
print("Python version:", sys.version)


In [ ]:
%pip install openai python-dotenv graphviz colorama


In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")


In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)


## The Agent Class


First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.


In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")


You can also associate tools with the agent. Let's create a tool for writing some string into a file.


In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")


## The Crew for J2: JSSP Advanced (Multi-Objective)


In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Job & Machine Setup Agent ---- #
    JM_Agent = Agent(
        name="Job & Machine Setup Agent",
        backstory="You define jobs, machines, processing times, and additional constraints for multi-objective JSSP.",
        task_description="Set up advanced JSSP instance with jobs, machines, processing times, and additional constraints (due dates, job priorities, setup times).",
        task_expected_output="Structured JSSP data: 1) Jobs with operations, due dates, and priorities. 2) Machines with setup times. 3) Processing times and constraints."
    )
    
    # ---- Multi-Objective Planning Agent ---- #
    MP_Agent = Agent(
        name="Multi-Objective Planning Agent",
        backstory="You plan scheduling considering multiple objectives: makespan, tardiness, machine idle time, and workload balance.",
        task_description="Create schedule optimizing multiple objectives: minimize makespan, minimize total tardiness, minimize machine idle time, and balance workload across machines.",
        task_expected_output="Multi-objective schedule with tradeoff analysis and objective values (makespan, tardiness, idle time, workload balance)."
    )
    
    # ---- Pareto Optimization Agent ---- #
    PO_Agent = Agent(
        name="Pareto Optimization Agent",
        backstory="You optimize the schedule to find Pareto-optimal solutions balancing multiple objectives.",
        task_description="Optimize schedule to find Pareto-optimal solutions that balance makespan, tardiness, idle time, and workload balance objectives.",
        task_expected_output="Pareto-optimal schedule with objective tradeoffs and multiple solution alternatives."
    )
    
    # ---- Performance Evaluation Agent ---- #
    PE_Agent = Agent(
        name="Performance Evaluation Agent",
        backstory="You evaluate schedule performance across all objectives and constraints.",
        task_description="Evaluate schedule performance: calculate makespan, total tardiness, machine utilization, workload balance, and constraint satisfaction.",
        task_expected_output="Performance evaluation report with metrics for all objectives and constraint satisfaction status."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './j2_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    JM_Agent >> MP_Agent >> PO_Agent >> PE_Agent >> Writer_agent


In [ ]:
crew.run()
